In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from pylab import mpl
import pandas as pd
from sklearn.model_selection import train_test_split
from math import sqrt 
import statsmodels.api as sm
import warnings
warnings.simplefilter('ignore', np.RankWarning)
import os
import re

In [5]:
#数据预处理 处理重复值运行一遍就行，第二遍会保存
path =r'E:/电热毯/'
files_list = os.listdir(path)
for file in files_list:
    df=pd.read_excel(path+"\\"+file, sheet_name='US')
    df['父体'].fillna(df['ASIN'], inplace = True)
    df = df[df['月销量'] != 0]
    df.drop_duplicates(subset=['品牌','月销量'],inplace = True)
    df.drop_duplicates(subset='父体',inplace = True)
    writer = pd.ExcelWriter(file)
    df.to_excel('E:/电热毯/{}'.format(file),index = False)
    writer.save()

D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook cont

D:\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
col=["p(5)","p(10)","p(20)","p(25)","p(50)","p(100)","p(200)","p(300)"]
dc=pd.DataFrame(columns=col)
'''
局部加权线性回归和局部加权多项式回归模型
在创建对象时，传入参数
i 代表文件夹里第几个文件
M 代表阶数，如果M=1就是局部加权线性回归，如果M=2就是局部加权二项式回归，数据量大二项式回归RMSE稍微更加优异点
如果数据太少，只有2条的情况还是局部加权线性回归更好，其实效果都挺好随便用哪个
sigma 就是非参数回归中的窗宽也就是一个惩罚系数，这个用到矩阵的取逆操作，所以sigma如果太小会报错，一般从0.5开始
sigma一搬来说越小拟合效果越好，但是不能太小会过拟合，看画图效果就可以了
'''

class openfile:
    def __init__(self,i,M,sigma):
        self.file_path = 'E:/电热毯/'
        self.file_list = os.listdir(self.file_path)
        self.file_name=self.file_list[i]
        self.df = pd.read_excel(self.file_path+self.file_list[i])
        self.y=self.df['月销量']
        if self.y.isnull().any()==True:
            self.y=self.y.fillna(0)
            #print("存在缺失")
        self.n=len(self.y)
        self.x = np.linspace(1,self.n,self.n)
        self.M=M
        self.sigma=sigma

    def local_polynomial_kernel_regression(self):#计算拟合值的功能，返回拟合的y
        y=[]
        kernel = lambda x,c,sig: np.exp(-(x-x[c])**2/(2*(sig**2)))/(np.sqrt(2*np.pi)*sig)
        for i in range(self.n):
            weight = kernel(self.x,i,self.sigma)#计算邻域各位置的权值
            W = np.diag(weight)
            design_matrix = np.asmatrix(np.ones(self.n)).T
            for j in range(1,self.M+1):
                arr = np.asmatrix(np.power(self.x,j)).T
                design_matrix  = np.concatenate((design_matrix ,arr),axis=1)   
            #print("design_matrix",design_matrix.shape)
            coef = np.asarray((design_matrix.T*W*design_matrix).I*(design_matrix.T*W*(np.asmatrix(self.y).T)))   
            #print("coef",coef.shape)
            if self.M==1:#局部线性回归
                y.append(coef[1]*self.x[i] + coef[0])
            if self.M==2:#局部二阶多项式回归     
                y.append(coef[2]*self.x[i]*self.x[i] + coef[1]*self.x[i] + coef[0])
        return y    
    
    def RMSE(self,M,sigma):#计算模型Rmse
        e=0
        n=len(self.x)
        y_rec=self.local_polynomial_kernel_regression()
        for i in range(n):
            e+=(y_rec[i]-self.y[i])**2
        Rmse=sqrt(e)/n    
        return Rmse
    def plot(self):#画图
        plt.figure(dpi=180,figsize=(6,3))
        plt.plot(self.x,self.y,'yx')
        y_rec= self.local_polynomial_kernel_regression()
        plt.plot(self.x,y_rec,'r')
    def getvalue(self):#计算所需位置的拟合值，比如头部，拐点之类的，最后输出一个dataframe
        y=self.local_polynomial_kernel_regression()
        string=str(self.file_name)
        p1 = re.compile(r'[-](.*?)[-]', re.S)
        time=re.findall(p1, string)[1]
        #col=["p(5)","p(10)","p(20)","p(25)","p(50)","p(100)","p(200)","p(300)"]
        #dc=pd.DataFrame(columns=col)
        p=[]
        for i in col:
            string=i
            p1 = re.compile(r'[(](.*?)[)]', re.S)
            num=int(re.findall(p1, string)[0])
            if num>=self.n:
                p.append(0)
            else:
                p.append(int(y[num]))
        dc.loc[time]=p
        return dc
        

In [11]:
a=openfile(1,2,0.3)
a.local_polynomial_kernel_regression()

design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 3)
coef (3, 1)
design_matrix (138, 

[array([627.00073242]),
 array([621.99999991]),
 array([482.00000002]),
 array([344.00000014]),
 array([311.99999991]),
 array([279.00000006]),
 array([277.9999999]),
 array([244.00000006]),
 array([239.00000004]),
 array([213.00000002]),
 array([208.99999967]),
 array([205.99999962]),
 array([197.00000033]),
 array([193.00000043]),
 array([157.99999933]),
 array([154.00000095]),
 array([141.00000067]),
 array([129.00000034]),
 array([124.00000126]),
 array([118.99999957]),
 array([112.00000042]),
 array([106.9999988]),
 array([92.00000231]),
 array([85.99999779]),
 array([85.99999925]),
 array([78.9999979]),
 array([71.99999962]),
 array([71.99999832]),
 array([70.99999945]),
 array([54.99999917]),
 array([54.00000008]),
 array([52.99999905]),
 array([51.99999989]),
 array([46.99999742]),
 array([43.99999719]),
 array([39.00000198]),
 array([38.99999858]),
 array([37.99999782]),
 array([35.99999506]),
 array([34.99999905]),
 array([33.99999278]),
 array([34.00000443]),
 array([31.9999

In [7]:
file_path = 'E:/电热毯/'
file_list = os.listdir(file_path)
for i in range(len(file_list)):
    a=openfile(i,1,0.3)
    #a.plot()
    t=a.getvalue()


In [8]:
t

p(5) p(10) p(20) p(25) p(50) p(100) p(200) p(300)
2018.01   608   478   239   196    87      0      0      0
2018.02   279   209   112    79    24      7      0      0
2018.03   165   122    72    61    27      0      0      0
2018.04   255   173    93    72    32      0      0      0
2018.05   143    80    38    31    11      0      0      0
2018.06   135    64    39    28    10      0      0      0
2018.07     0     0     0     0     0      0      0      0
2018.08   266   163    93    76    25      0      0      0
2018.09   530   422   238   210    70     19      0      0
2018.10  1149   844   503   409   217     43      0      0
2018.11  2184   978   673   498   252     69      9      0
2018.12  4289  2061  1591  1173   595    127     24      0
2019.01  1197   955   603   438   181     54      7      0
2019.02   941   657   410   319   150     49      1      0
2019.03   778   541   312   286   142     63      7      0
2019.04  1185   919   661   589   268    107     16      0
2019.05  1412   998   752   691   350    157     32      2
2019.06   928   717   321   281    98      0      0      0
2019.07   975   407   140   115    22      3      0      0
2019.08   995   649   297   223    69      4      0      0
2019.09  2461  1252   481   394    93      7      0      0
2019.10  5495  2149   889   654   181     13      0      0
2019.11  2271  2096  1242   880   291     62      0      0
2019.12  5518  2737  1169  1026   265     42      0      0
2020.01  4105  1784   713   528   202     15      0      0
2020.02  2586  1312   397   331   118      3      0      0
2020.03  1903  1038   319   205    57      2      0      0
2020.04  1484   842   319   215    56      2      0      0
2020.05   837   395   118    86    24      0      0      0
2020.06   419   203    91    69    15      0      0      0
2020.07   617   290   155    94    26      0      0      0
2020.08   879   392   178    99    22      0      0      0
2020.09  4107  2240  1076   656   117     14      0      0
2020.10  5009  3055  1694  1402   459     84      0      0
2020.11  8303  5301  2083  1842   674    140      1      0
2020.12  9049  7166  4776  3482  1460    261     14      0
2021.01  5374  4310  2517  1823   898    205     13      0
2021.02  4554  2740  1825  1394   568    153      3      0
2021.03  2605  1103   328   262   126     29      0      0
2021.04  2053   840   390   270    90     13      0      0
2021.05  1195   867   464   341    64     10      0      0